# VLM

# RAG

- Храним данные в Chroma, collection_name="dota_knowledge_1.3"
- На данный момент там небольшая справка про каждого из героев доты с wikifandom, данные повторяются

## Собираем данные

### Качаем библиотеки

In [ ]:
pip install langchain_chroma langchain_community

KeyboardInterrupt: 

In [ ]:
import nltk
import json

from pathlib import Path
from pprint import pprint

import subprocess
import os

from langchain_community.document_loaders import JSONLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader

from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter
import chromadb
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import BSHTMLLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.text_splitter import RecursiveCharacterTextSplitter

### Дообучение энкодера под домен -- coming soon

In [ ]:
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader


train_examples = #?

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

model = SentenceTransformer('all-MiniLM-L6-v2')

train_loss = losses.MultipleNegativesRankingLoss(model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    output_path='./dota2-encoder'
)

### Парсим данные

In [ ]:
os.makedirs('dota_wget', exist_ok=True)

In [ ]:
dota_heroes = dota_heroes = [
    # Strength Heroes
    "Abaddon",
    "Alchemist",
    "Axe",
    "Bristleback",
    "Centaur_Warrunner",
    "Chaos_Knight",
    "Clockwerk",
    "Dawnbreaker",
    "Doom",
    "Dragon_Knight",
    "Earth_Spirit",
    "Earthshaker",
    "Elder_Titan",
    "Huskar",
    "Io",
    "Kunkka",
    "Legion_Commander",
    "Lifestealer",
    "Lycan",
    "Magnus",
    "Marci",
    "Mars",
    "Night_Stalker",
    "Ogre_Magi",
    "Omniknight",
    "Phoenix",
    "Pudge",
    "Sand_King",
    "Slardar",
    "Snapfire",
    "Sven",
    "Tidehunter",
    "Timbersaw",
    "Tiny",
    "Treant_Protector",
    "Tusk",
    "Underlord",
    "Undying",
    "Wraith_King",

    # Agility Heroes
    "Anti-Mage",
    "Arc_Warden",
    "Bloodseeker",
    "Bounty_Hunter",
    "Clinkz",
    "Drow_Ranger",
    "Ember_Spirit",
    "Faceless_Void",
    "Gyrocopter",
    "Hoodwink",
    "Juggernaut",
    "Lone_Druid",
    "Luna",
    "Medusa",
    "Meepo",
    "Mirana",
    "Monkey_King",
    "Morphling",
    "Naga_Siren",
    "Nyx_Assassin",
    "Pangolier",
    "Phantom_Assassin",
    "Phantom_Lancer",
    "Razor",
    "Riki",
    "Shadow_Fiend",
    "Slark",
    "Sniper",
    "Spectre",
    "Templar_Assassin",
    "Terrorblade",
    "Troll_Warlord",
    "Ursa",
    "Vengeful_Spirit",
    "Venomancer",
    "Viper",
    "Weaver",

    # Intelligence Heroes
    "Ancient_Apparition",
    "Crystal_Maiden",
    "Dark_Seer",
    "Dark_Willow",
    "Death_Prophet",
    "Disruptor",
    "Enchantress",
    "Enigma",
    "Grimstroke",
    "Invoker",
    "Jakiro",
    "Keeper_of_the_Light",
    "Leshrac",
    "Lich",
    "Lina",
    "Lion",
    "Nature's_Prophet",
    "Necrophos",
    "Oracle",
    "Outworld_Destroyer",
    "Puck",
    "Pugna",
    "Queen_of_Pain",
    "Rubick",
    "Shadow_Demon",
    "Shadow_Shaman",
    "Silencer",
    "Skywrath_Mage",
    "Storm_Spirit",
    "Techies",
    "Tinker",
    "Visage",
    "Void_Spirit",
    "Warlock",
    "Windranger",
    "Winter_Wyvern",
    "Witch_Doctor",
    "Zeus",

    # Universal Heroes
    "Muerta"
]

In [ ]:
def download_with_wget(hero, item=''):
  commands = [
        'wget', f'https://dota2.fandom.com/wiki/{hero}/{item}',
        '-O', f'dota_wget/{hero}{item}.html',
        '--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    ]

  try:
    result = subprocess.run(commands, capture_output=True, text=True)
    if result.returncode != 0:
        print(f'https://dota2.fandom.com/wiki/{hero}/{item} has an error {result.returncode}')
  except subprocess.TimeoutExpired:
        print(f"⏰ Таймаут при скачивании {hero}{item}")
        return False
  except Exception as e:
        print(f"❌ Неожиданная ошибка: {e}")
        return False

In [99]:
traits = ['Guide', 'Equipment', 'Talents']

In [ ]:
import time

for hero in dota_heroes:
  for item in traits:
    download_with_wget(hero, item)
    time.sleep(4.3)

In [ ]:
for hero in dota_heroes:
#  for item in ['/Guide', '/Equipment', '/Talents']:
    download_with_wget(hero)

In [112]:
def merge_text_files(file_paths, output_file):
    merged_content = ""

    for i, file_path in enumerate(file_paths):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                merged_content += content
                print(f"Добавлен: {file_path}")

        except Exception as e:
            print(f"Ошибка чтения {file_path}: {e}")

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(merged_content)

    return output_file

for hero in dota_heroes:
    merge_text_files([f'/content/dota_wget/{hero}.html', f'/content/dota_wget/{hero}Guide.html',
                     f'/content/dota_wget/{hero}Equipment.html', f'/content/dota_wget/{hero}Talents.html'],
                     f'/content/dota_wget/{hero}Total.html')

✅ Добавлен: /content/dota_wget/Abaddon.html
✅ Добавлен: /content/dota_wget/AbaddonGuide.html
✅ Добавлен: /content/dota_wget/AbaddonEquipment.html
✅ Добавлен: /content/dota_wget/AbaddonTalents.html
📄 Объединенный файл сохранен: /content/dota_wget/AbaddonTotal.html
📊 Размер: 1242667 символов
✅ Добавлен: /content/dota_wget/Alchemist.html
✅ Добавлен: /content/dota_wget/AlchemistGuide.html
✅ Добавлен: /content/dota_wget/AlchemistEquipment.html
✅ Добавлен: /content/dota_wget/AlchemistTalents.html
📄 Объединенный файл сохранен: /content/dota_wget/AlchemistTotal.html
📊 Размер: 1240504 символов
✅ Добавлен: /content/dota_wget/Axe.html
✅ Добавлен: /content/dota_wget/AxeGuide.html
✅ Добавлен: /content/dota_wget/AxeEquipment.html
✅ Добавлен: /content/dota_wget/AxeTalents.html
📄 Объединенный файл сохранен: /content/dota_wget/AxeTotal.html
📊 Размер: 1295735 символов
✅ Добавлен: /content/dota_wget/Bristleback.html
✅ Добавлен: /content/dota_wget/BristlebackGuide.html
✅ Добавлен: /content/dota_wget/Brist

## Делаем RAG

### Кодируем их и грузим в Chroma

In [194]:
class DotaVectorStore:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.client = chromadb.PersistentClient(path="./dota_db_1.3")
        self.embedding_model = SentenceTransformer(model_name)
        self.doc_counter = 0
        self.collection = self.client.get_or_create_collection(
            name="dota2_knowledge_1.3",
            metadata={"description": "Dota 2 game knowledge base"}
        )
    def _embed_documents(self, texts):
        return self.embedding_model.encode(texts).tolist()

    def _embed_query(self, text):
        return self.embedding_model.encode([text]).tolist()[0]

    def add_document(self, document_html, metadatas=None):
        loader = BSHTMLLoader(document_html)
        data = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=700, chunk_overlap=100)
        docs = text_splitter.split_documents(data)
        texts = [doc.page_content for doc in docs]
        embeddings = self._embed_documents(texts)
        start_id = self.doc_counter
        ids = [f"doc_{start_id + i}" for i in range(len(texts))]
        self.doc_counter += len(texts)
        self.collection.add(
            documents=texts,
            embeddings=embeddings,
            metadatas=metadatas,
            ids=ids
        )

        print(f"✅ Добавлено {len(texts)} документов")

    def search(self, query, n_results=5):
        query_embedding = self._embed_query(query)
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results
        )

        return results

In [195]:
rag_store = DotaVectorStore()

In [196]:
for hero in dota_heroes:
    rag_store.add_document(f'/content/dota_wget/{hero}Total.html')

✅ Добавлено 67 документов


✅ Добавлено 71 документов


✅ Добавлено 66 документов


✅ Добавлено 66 документов
✅ Добавлено 68 документов


✅ Добавлено 65 документов


...

✅ Добавлено 63 документов
✅ Добавлено 65 документов


✅ Добавлено 63 документов


✅ Добавлено 71 документов


✅ Добавлено 64 документов


✅ Добавлено 72 документов
✅ Добавлено 58 документов


### Проверим как работает RAG

In [198]:
def list_all_documents(collection_name="dota2_knowledge_1.3", persist_directory="./dota_db_1.3"):
    import chromadb

    client = chromadb.PersistentClient(path=persist_directory)
    collection = client.get_collection(collection_name)

    results = collection.get()

    print(f"📚 Коллекция: {collection_name}")
    print(f"📄 Всего документов: {len(results['ids'])}")
    print("=" * 60)

    for i, (doc_id, document, metadata) in enumerate(zip(
        results['ids'],
        results['documents'],
        results['metadatas']
    )):
        print(f"\n{i+1}. 🆔 ID: {doc_id}")
        print(f"   🏷️  Метаданные: {metadata}")
        print(f"   📝 Превью: {document[:100]}...")
        print("-" * 50)

    return results

documents_info = list_all_documents()

📚 Коллекция: dota2_knowledge_1.3
📄 Всего документов: 7776

1. 🆔 ID: doc_0
   🏷️  Метаданные: None
   📝 Превью: --- Документ 1: /content/dota_wget/Abaddon.html ---


Abaddon - Dota 2 Wiki

		Sign In	

		Register	...
--------------------------------------------------

2. 🆔 ID: doc_1
   🏷️  Метаданные: None
   📝 Превью: Twitter


Facebook


Community

 


Wiki Twitter


Wiki Steam Group


Wiki Discord


Dota 2 Reddit

...
--------------------------------------------------

3. 🆔 ID: doc_2
   🏷️  Метаданные: None
   📝 Превью: Game modes


Events


Patches


Versions


Console


Hero stats table


Cosmetics

 


Equipment


G...
--------------------------------------------------

4. 🆔 ID: doc_3
   🏷️  Метаданные: None
   📝 Превью: History										

											Purge										

											Talk (31)										


Hero
 
Strategy
 
Eq...
--------------------------------------------------

5. 🆔 ID: doc_4
   🏷️  Метаданные: None
   📝 Превью: L1

⤷ + Base Attributes


L15

⤷ + Attribute Gain


L25



In [203]:
pip install langchain_core

In [222]:
rag_store.search('Who is Alchemist', 3)

{'ids': [['doc_119', 'doc_101', 'doc_111']],
 'embeddings': None,
 'documents': [["Manta Style is excellent for sieging buildings and taking waves, particularly in conjunction with  Radiance. It also provides Alchemist with a much needed disjoint mechanism.\n Mjollnir increases Alchemist's attack speed greatly, and also gives a useful Static Charge to protect Alchemist or his teammates. Its lightning attack will also trigger often under Chemical Rage.\n Battle Fury allows Alchemist to farm much quicker.\n Blade Mail forces enemies to think twice about fighting back against Alchemist.\n Abyssal Blade goes well with Alchemist's high attack speed, and helps greatly in securing an enemy. It also gives bonus attack damage and greater survivability providing bonus HP and blocking damage from incoming attacks.\n Heaven's Halberd gives some strength, evasion, status resistance, and also gives the ability to disarm a threatening enemy.\n Heart of Tarrasque gives Alchemist a massive amount of he